In [ ]:
# import tensorflow 
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import to_categorical

# # Load the CIFAR-10 dataset
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()

# # Normalize the pixel values to be between 0 and 1
# x_train = x_train / 255.0
# x_test = x_test / 255.0

# # Convert the labels to one-hot encoded vectors
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# # Create a convolutional neural network
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(10, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model on the training data
# model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# # Evaluate the model on the testing data
# loss, accuracy = model.evaluate(x_test, y_test)

# print(f"Test accuracy: {accuracy}")

In [ ]:
def func(a):								
    for i in range(n):
        a=np.random.randn(n)
        b=np.random.randn(n)
        c=np.dot(a,b)

@jit(target_backend='cuda')
def func2(a):
    for i in range(n):
        A=np.random.randn(n)
        B=np.random.randn(n)
        C=np.dot(A,B)

if __name__=="__main__":
    n=100000
    a=np.random.randn(n)
    # print(a)
    start = timer()
    func(a)
    print("without GPU:", timer()-start)	
	
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)

In [ ]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):								
	for i in range(1000000):
		a[i]+= 1	

# function optimized to run on gpu
@jit(target_backend='cuda')						
def func2(a):
	for i in range(1000000):
		a[i]+= 1
if __name__=="__main__":
	n = 1000000		
	a = np.ones(n, dtype = np.float64)
	
	start = timer()
	func(a)
	print("without GPU:", timer()-start)	
	
	start = timer()
	func2(a)
	print("with GPU:", timer()-start)


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 14 15:36:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA T1000 8GB             WDDM  | 00000000:01:00.0  On |                  N/A |
| 34%   44C    P0              N/A /  50W |   1003MiB /  8192MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
name = "bigscience/bloom-3b"
text = "Hello my name is"
max_new_tokens = 20

def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [11]:
from transformers import pipeline

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

name = "bigscience/bloom-3b"
text = "Hello my name is"
max_new_tokens = 20


tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name)
pipe = pipeline(model=model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)
output = pipe(text)
print(output)



In [9]:
pipe(text)

TypeError: 'str' object is not callable

In [1]:
import numpy as np
from datasets import Dataset


seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, (dataset_size)),
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [2]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [6]:
print_gpu_utilization()

GPU memory occupied: 1128 MB.


In [8]:
import torch


torch.ones((1, 1)).to("cuda")
print_gpu_utilization()

AssertionError: Torch not compiled with CUDA enabled

In [9]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased").to("cuda")
print_gpu_utilization()

c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\matan.s\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 from transformers import AutoModelForSequenceClassification                                  │
│   2                                                                                              │
│   3                                                                                              │
│ ❱ 4 model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased").to("cud     │
│   5 print_gpu_utilization()                                                                      │
│   6                                                                                              │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\modeling_utils.py:1886 in  │
│ to                                                                                               │
│                                                                                                  │
│   1883 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1884 │   │   │   )                                                                             │
│   1885 │   │   else:                                                                             │
│ ❱ 1886 │   │   │   return super().to(*args, **kwargs)                                            │
│   1887 │                                                                                         │
│   1888 │   def half(self, *args):                                                                │
│   1889 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\torch\nn\modules\module.py:1145 in to   │
│                                                                                                  │
│   1142 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│   1143 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│   1144 │   │                                                                                     │
│ ❱ 1145 │   │   return self._apply(convert)                                                       │
│   1146 │                                                                                         │
│   1147 │   def register_full_backward_pre_hook(                                                  │
│   1148 │   │   self,                                                                             │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\torch\nn\modules\module.py:797 in       │
│ _apply                                                                                           │
│                                                                                                  │
│    794 │                                                                                         │
│    795 │   def _apply(self, fn):                                                                 │
│    796 │   │   for module in self.children():                                                    │
│ ❱  797 │   │   │   module._apply(fn)                                                             │
│    798 │   │                                                                                     │
│    799 │   │   def compute_should_use_set_data(tensor, tens